In [6]:
import sys
sys.path.append("/Users/hhg/Research/c_lib/")
import illustris_python as il
import pandas as pd
from astropy.table import Table
import h5py

%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [8]:
basePath_hydro = '/Users/hhg/Research/baryon_MassProfile/catalog/TNG100-3'
basePath_dmo   = '/Users/hhg/Research/baryon_MassProfile/catalog/TNG100-3-dark'

In [20]:
snap = 94

### 1. loading group catalog

In [7]:
fields = ['GroupFirstSub','GroupPos','Group_M_Mean200','Group_R_Mean200','GroupNsubs','GroupMass'] #'GroupMassType'

In [9]:
halo_dmo   = il.groupcat.loadHalos(basePath_dmo, snap, fields=fields)

In [11]:
halo_hydro = il.groupcat.loadHalos(basePath_hydro, snap, fields=fields)

### 2. loading snapshot catalog info

In [192]:
dm_hydro_pos = il.snapshot.loadSubset(basePath_hydro, snap, 'dm', fields=["Coordinates"])

In [193]:
gas_hydro_pos = il.snapshot.loadSubset(basePath_hydro, snap, 'gas', fields=["Coordinates"])

In [194]:
dm_hydro_pos.shape

(94196375, 3)

In [195]:
gas_hydro_pos.shape

(89037187, 3)

In [199]:
all_pos = np.vstack((dm_hydro_pos, gas_hydro_pos))
all_pos.shape

(183233562, 3)

In [200]:
94196375+89037187

183233562

In [12]:
fields=["Coordinates"]
dm_hydro_pos = il.snapshot.loadSubset(basePath_hydro, snap, 'dm', fields=fields)
dm_hydro_pos = dm_hydro_pos/1000.     # change unit to Mpc/h

dm_dmo_pos = il.snapshot.loadSubset(basePath_dmo, snap, 'dm', fields=fields)
dm_dmo_pos = dm_dmo_pos/1000.         # change unit to Mpc/h

In [75]:
fields=["Coordinates"]
gas_hydro_pos = il.snapshot.loadSubset(basePath_hydro, snap, 'gas', fields=fields)
gas_hydro_pos = gas_hydro_pos/1000.     # change unit to Mpc/h

### 3. load "subhalo_matching_to_dark.hdf5"

In [17]:
filename = basePath_hydro+'/subhalo_matching_to_dark.hdf5'
hf = h5py.File(filename,'r')

In [18]:
hf.keys()

<KeysViewHDF5 ['Header', 'Snapshot_0', 'Snapshot_1', 'Snapshot_10', 'Snapshot_11', 'Snapshot_12', 'Snapshot_13', 'Snapshot_14', 'Snapshot_15', 'Snapshot_16', 'Snapshot_17', 'Snapshot_18', 'Snapshot_19', 'Snapshot_2', 'Snapshot_20', 'Snapshot_21', 'Snapshot_22', 'Snapshot_23', 'Snapshot_24', 'Snapshot_25', 'Snapshot_26', 'Snapshot_27', 'Snapshot_28', 'Snapshot_29', 'Snapshot_3', 'Snapshot_30', 'Snapshot_31', 'Snapshot_32', 'Snapshot_33', 'Snapshot_34', 'Snapshot_35', 'Snapshot_36', 'Snapshot_37', 'Snapshot_38', 'Snapshot_39', 'Snapshot_4', 'Snapshot_40', 'Snapshot_41', 'Snapshot_42', 'Snapshot_43', 'Snapshot_44', 'Snapshot_45', 'Snapshot_46', 'Snapshot_47', 'Snapshot_48', 'Snapshot_49', 'Snapshot_5', 'Snapshot_50', 'Snapshot_51', 'Snapshot_52', 'Snapshot_53', 'Snapshot_54', 'Snapshot_55', 'Snapshot_56', 'Snapshot_57', 'Snapshot_58', 'Snapshot_59', 'Snapshot_6', 'Snapshot_60', 'Snapshot_61', 'Snapshot_62', 'Snapshot_63', 'Snapshot_64', 'Snapshot_65', 'Snapshot_66', 'Snapshot_67', 'Snapsh

In [19]:
hf.get(u'Snapshot_%02d'%snap).keys()

<KeysViewHDF5 ['SubhaloIndexDark_LHaloTree', 'SubhaloIndexDark_SubLink']>

Data format: a single file subhalo_matching_to_dark.hdf5 for each simulation. Each contains one hundred groups named Snapshot_N. Within each group are two arrays, each with the same size, equal to the number of subhalos at the corresponding snapshot. The two arrays provide the results of two different matching algorithms, and should for the most part provide the same answer. Each array gives integer indices, whose value is the corresponding index of the matched subhalo in the DM only run. If no suitable match exists, a value of -1 is present for that subhalo index.

The first array SubhaloIndexDark_LHaloTree is based on the LHaloTree matching algorithm. The matching is bidirectional, i.e. TNG <-> DMO. In each base, the best subhalo candidate is chosen as that with the largest number of matching DM particles (α=0
). Only if the candidate in each direction agrees (bijective), then these matches are saved.

The second array SubhaloIndexDark_SubLink is based on the SubLink weighting algorithm. The direction of the matching is TNG -> DMO, i.e. for each subhalo in the baryonic physics box a best match is found in the DMO run.

Future: The second two types of matching catalogs (between different resolution levels, and between TNG100 and original Illustris), will be available for selected snapshots in the future, following Lovell+ (2018).

In [21]:
ID_matched = np.array(hf[u'Snapshot_%02d'%snap]['SubhaloIndexDark_LHaloTree'])

In [23]:
ID_matched[0:100]

array([    0,     1,     2,     3,     4,     9,     8,    -1,     5,
           7,     6,    -1,    13,    -1,    -1,    10,    -1,    11,
          18,    15,    19,    22,    -1,    17,    -1,    23,    14,
          20,    25, 25773,    27,    28,    -1,    -1,    34,    -1,
          21,    -1,    30,    31,    50,    36,    -1,    -1,    32,
          -1,    33,    26,    35,    37,    38,    47,    -1,    55,
          -1,    -1,    76,    45,    42,    44,    40,    64,    52,
          54,    48,    39,    -1,    59,    -1,    51,    57,    56,
          -1,    -1,    -1,    -1,    -1,    61,    -1,    62,    69,
          -1,    63,    -1,    83,    90,    80,    85,    65,    92,
          -1,    66,    -1,    95,    -1,    87,    81,    78,    97,
          71], dtype=int32)

In [24]:
ID_matched = np.array(hf[u'Snapshot_%02d'%snap]['SubhaloIndexDark_SubLink'])

In [25]:
ID_matched[0:100]

array([    0,     1,     2,     3,     4,     9,     8,    46,     5,
           7,     6,     0,    13,     0,     0,    10,    29,    11,
          18,    15,    19,    22,    16,    17,     1,    23,    14,
          20,    25, 25773,    27,    28,     0,     0,    34,     0,
          21,    58,    30,    31,     1,    36,    24,     0,    32,
          75,    33,    26,    35,    37,    38,    47,     1,    55,
           1,     0,    76,    45,    42,    44,    40,    64,    52,
          54,    48,    39,    60,    59,    82,    51,    57,    56,
          70,     0,     0,     0,     0,    61,     0,    62,    69,
          68,    63,     0,     0,    90,    80,    85,    65,    92,
          73,    66,     0,    95,     0,    87,    81,    78,    97,
          71], dtype=int32)

In [187]:
%run ../code/hydrosim_kit.py
ID_matched2 = load_subhalo_matching_to_dark(basePath_hydro, 94)

available keys: <KeysViewHDF5 ['SubhaloIndexDark_LHaloTree', 'SubhaloIndexDark_SubLink']>


In [190]:
ID_matched2

array([ 0,  1,  2, ..., -1, -1, -1], dtype=int32)

### 4. test class TNG_info

In [201]:
%run ../code/load_catalog.py

In [202]:
sim_info = TNG_info(basePath="/Users/hhg/Research/baryon_MassProfile/catalog/TNG100-3-dark", snap=94)

In [203]:
sim_info.header_groupCat

{'BoxSize': 75000.0,
 'FlagDoubleprecision': 0,
 'Git_commit': b'd203ec8b07c7e2bdda5f608aa0babea46d603699',
 'Git_date': b'Thu Apr 7 14:14:27 2016 +0200',
 'HubbleParam': 0.6774,
 'Ngroups_ThisFile': 4393,
 'Ngroups_Total': 95990,
 'Nids_ThisFile': 11382173,
 'Nids_Total': 45769534,
 'Nsubgroups_ThisFile': 11056,
 'Nsubgroups_Total': 117157,
 'NumFiles': 4,
 'Omega0': 0.3089,
 'OmegaLambda': 0.6911,
 'Redshift': 0.058507322794512984,
 'Time': 0.9447265771954694}

In [204]:
sim_info.header_snapCat

{'BoxSize': 75000.0,
 'Composition_vector_length': 0,
 'Flag_Cooling': 0,
 'Flag_DoublePrecision': 0,
 'Flag_Feedback': 0,
 'Flag_Metals': 0,
 'Flag_Sfr': 0,
 'Flag_StellarAge': 0,
 'Git_commit': b'd203ec8b07c7e2bdda5f608aa0babea46d603699',
 'Git_date': b'Thu Apr 7 14:14:27 2016 +0200',
 'HubbleParam': 0.6774,
 'MassTable': array([0.        , 0.03839801, 0.        , 0.        , 0.        ,
        0.        ]),
 'NumFilesPerSnapshot': 4,
 'NumPart_ThisFile': array([       0, 23570109,        0,        0,        0,        0],
       dtype=int32),
 'NumPart_Total': array([       0, 94196375,        0,        0,        0,        0],
       dtype=uint32),
 'NumPart_Total_HighWord': array([0, 0, 0, 0, 0, 0], dtype=uint32),
 'Omega0': 0.3089,
 'OmegaBaryon': 0.0486,
 'OmegaLambda': 0.6911,
 'Redshift': 0.058507322794512984,
 'Time': 0.9447265771954694,
 'UnitLength_in_cm': 3.085678e+21,
 'UnitMass_in_g': 1.989e+43,
 'UnitVelocity_in_cm_per_s': 100000.0}

In [186]:
sim_info.Nptls

{'dm': 94196375, 'gas': 0, 'star': 0, 'bh': 0}

### 5. test class TNGsnap_info

In [230]:
%run ../code/load_catalog.py

In [231]:
basePath = "/Users/hhg/Research/baryon_MassProfile/catalog/TNG100-3-dark"
snap = 94

In [232]:
snapInfo = TNGsnap_info(basePath, snap)

loading snapshot ptl: dm


In [233]:
snapInfo.ptl_info['pos'].shape

(94196375, 3)

### 6. test class TNGgroup_info

In [318]:
%run ../code/load_catalog.py

basePath = "/Users/hhg/Research/baryon_MassProfile/catalog/TNG100-3"
snap = 94

In [319]:
TNGhalo = TNGgroup_info(basePath, snap)

available keys: <KeysViewHDF5 ['SubhaloIndexDark_LHaloTree', 'SubhaloIndexDark_SubLink']>


In [322]:
TNGhalo.halo_cat

{'GroupFirstSub': array([     0,    420,    797, ..., 120263, 120264, 120265], dtype=int32),
 'GroupPos': array([[  832.96014, 26403.248  , 18410.014  ],
        [19993.701  , 48258.637  , 48648.336  ],
        [10804.087  , 50909.492  , 47557.23   ],
        ...,
        [64520.195  , 47187.598  , 15187.027  ],
        [58658.555  , 13320.293  , 55673.53   ],
        [60744.94   ,  3213.2454 , 73041.836  ]], dtype=float32),
 'Group_M_Mean200': array([3.0557238e+04, 3.4887852e+04, 2.8777768e+04, ..., 7.4982941e-01,
        6.8547320e-01, 6.4713508e-01], dtype=float32),
 'Group_R_Mean200': array([1620.4757  , 1693.6893  , 1588.462   , ...,   47.091873,
          45.703583,   44.834366], dtype=float32),
 'GroupNsubs': array([420, 377, 353, ...,   1,   1,   1], dtype=int32),
 'GroupMass': array([3.2137162e+04, 3.1796176e+04, 2.6912441e+04, ..., 8.3384454e-01,
        7.9033959e-01, 7.9189974e-01], dtype=float32),
 'ID_FoFhalo': array([     0,      1,      2, ..., 126586, 126587, 126588]),

### 7. test TNGmatched_groupCat

In [365]:
%run ../code/load_catalog.py

basePath_hydro = "/Users/hhg/Research/baryon_MassProfile/catalog/TNG100-3"
basePath_dmo = "/Users/hhg/Research/baryon_MassProfile/catalog/TNG100-3-dark"
snap = 94

In [366]:
Cat_matched = TNGmatched_groupCat(basePath_hydro, basePath_dmo, snap)

available keys: <KeysViewHDF5 ['SubhaloIndexDark_LHaloTree', 'SubhaloIndexDark_SubLink']>


In [377]:
np.log10(np.max(Cat_matched.matched_halo_cat_hydro['GroupMass'])*1e10)

14.507007523495947

In [378]:
np.log10(np.min(Cat_matched.matched_halo_cat_hydro['GroupMass'])*1e10)

9.820875859305602

In [379]:
Cat_matched = TNGmatched_groupCat(basePath_hydro, basePath_dmo, snap, log10_Mhalo_range=[14.0, 15.])

available keys: <KeysViewHDF5 ['SubhaloIndexDark_LHaloTree', 'SubhaloIndexDark_SubLink']>


In [380]:
Cat_matched.matched_halo_cat_hydro['ID_FoFhalo']

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13])

In [381]:
Cat_matched.matched_halo_cat_dmo['ID_FoFhalo']

array([ 0,  1,  2,  3,  4,  5,  7,  8,  9,  6, 10, 11, 12, 13])

In [383]:
Cat_matched.matched_halo_cat_hydro['GroupPos'].shape

(14, 3)

In [384]:
Cat_matched.matched_halo_cat_dmo['GroupPos'].shape

(14, 3)